<a href="https://colab.research.google.com/github/alonsaguy/QAFKA/blob/main/main_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**QAFKA (Quantitative Analysis of Flourescent Kinetics Algorithm)**
##**Welcome to QAFKA jupyter notebook**
**Disclaimer:**

This notebook is based on the following paper: Automated analysis of fluorescence kinetics in single-molecule localization microscopy data reveals protein stoichiometry. [link to paper](google.com)

And source code found in: https://github.com/alonsaguy/QAFKA

Please also cite this original paper when using or developing this notebook.

##**Initialize Colab Session**
###**Mount your Google Drive**
To use this notebook on the data present in your Google Drive, you need to mount your Google Drive to this notebook.

Play the cell below to mount your Google Drive and follow the link. In the new browser window, select your drive and select 'Allow', copy the code, paste into the cell and press enter. This will give Colab access to the data on the drive.

Once this is done, your data are available in the *Files* tab on the top left of notebook.


In [ ]:
#@markdown ##Run this cell to connect your Google Drive to Colab

#@markdown * Click on the URL. 

#@markdown * Sign in your Google Account. 

#@markdown * Copy the authorization code. 

#@markdown * Enter the authorization code. 

#@markdown * Click on "Files" site on the right. Refresh the site. Your Google Drive folder should now be available here as "drive". 

#mounts user's Google Drive to Google Colab.

from google.colab import drive
drive.mount('/content/gdrive')

##**Intall QAFKA and dependencies**
In the next block we will import the relevant packages for QAFKA.

In [ ]:
!pip install tiffcapture
!pip install torch
!pip install ipympl

from datasets import *
from dataloaders import *
from neural_network import *
from trainers import *
from utils import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

print("Packages installation completed successfully")

##**Parameters Initiallization**
Now you should specify some important parameters for your run.

**numOfBins** - Specifies the number of bins in the histogram of the number of blinking events (default: 20)

**start/end_frame** - Specifies which frames of the experiment you would like to analyze. For example: start_frame = 0 and end_frame=1000 will cause QAFKA to analyze the experiment between the first frame and the 1000th frame.

**pixel_length** - Specifies the experiment's pixel size [nm]

**scale_size** - Specifies the resolution scaling. For example: scale_size = 3 and pixel_length = 150 [nm] would results in a reconstrcuted image with grid size of 50 [nm].

**emitters_size** - Specifies the emitters merging radius for detection. If two clusters would be located within this radius they would be considered as a single cluster.

**numOfClusters** - Specifies the number of simulated clusters in each simulated experiment

**file_names** - Specifies the names of the TIFF files (at least one experiment is required). For example: 'first_exp.tif'.

**qualityThreshold** - Specifies the minimal fitting score for the localization block. (default: 0.85)

In [ ]:
numOfBins = 20 #@param {type:"number"}
start_frame = 0 #@param {type:"number"}
end_frame = 2000 #@param {type:"number"}
pixel_length = 157# @param {type:"number"}
scale_size = 3 #@param {type:"number"}
merging_radius = 50 #@param {type:"number"}
file_names = 'D:\Project\data\CTLA4\mEos3.2.tif' #@param {type:"string"}
qualityThreshold = 0.85 #@param {type:"number"}
chop = [start_frame, end_frame]

##**Run Configuration**
**LoadData** - Determines if we want to load new experimental data (True) or we want to use an already loaded data (False).

**FilterBeads** - Determines if an additional beads filtration algorithm is needed for the experimental data.

**CreateSimulatedData** - Determines if we want to use the same training data as before (True) or we want to create new training set (False).

**TrainNet** - Determines if we want to train the neural network (True) or not (False). Usually it takes ~5 minutes to train the network.

**preTrainedModel** - Specifies the pre-trained model to load in case we do not want to train the net.

In [ ]:
LoadData = True #@param {type:"boolean"}
FilterBeads = False #@param {type:"boolean"}
CreateSimulatedData = True #@param {type:"boolean"}
TrainNet = True #@param {type:"boolean"}
preTrainedModel = 'model_final_gauss' #@param {type:"string"}

##**Analysis Pipeline**
Run this block to analyze the provided tiff stacks and generate number of blinking events histograms per experiment.

In [ ]:
if(chop[1]-chop[0]<2000):
    max_size = int((chop[1]-chop[0])/500)
else:
    max_size = int((chop[1]-chop[0])/1000)
    
resolution_nm = pixel_length/scale_size #[nm]

if(LoadData):
    trajectories, clusterCoordinations = [], []
    for i, file in enumerate(file_names):
        print("**** Analyzing Tiff number {} ****".format(i+1))
        # Load TIFF files and create data_set
        Data_Set = CreateDataSet(file, chop)
        
        # Segment the experiment before and after laser activation
        seg = segment(Data_Set, threshold=0.15, window_size=100)
        
        # Filter beads (if True)
        if(FilterBeads):
            Data_Set = Filter_beads(Data_Set)
        
        # Background noise cleaning
        Data_Set = clean_bg_noise(Data_Set, patch_length=5)
        
        # Clusters localization
        Max_Data_Set = CreateMaxDataSet(Data_Set, max_size, seg)
        DataThreshold, MaxThreshold = calc_threshold(Data_Set, Max_Data_Set)
        coordinates = LocalizeEmitters(Max_Data_Set, MaxThreshold, qualityThreshold, pixel_length, resolution_nm, merging_radius)
        
        # Create time traces for each cluster
        timeTraces = ExtractTimeTraces(Data_Set[seg:, :, :], coordinates, pixel_length, resolution_nm, qualityThreshold, DataThreshold, merging_radius)
        
        # Save the time traces and clusters locations of all experiments in a list
        trajectories.append(timeTraces)
        clusterCoordinations.append(coordinates)
        
        # The coordinations file would be saved as 'coordinated.npy'
    np.save('clusterCoordinations', clusterCoordinations)

    # Extract the features that would serve as the neural network's input
    X_test = feature_extraction(trajectories, DataThreshold, numOfBins)
else:
    # Load features of an already analyzed experiment
    X_test = LoadFinalDataSet()

print("Experimental Data was loaded successfully")

##**Plot Histograms**

In [ ]:
%matplotlib widget

for i in range(X_test.shape[0]):
    plt.figure()
    plt.plot(np.arange(1, numOfBins + 1), X_test[i])
    plt.xlabel('Bins')
    plt.ylabel('Counts')
    plt.xticks(np.arange(1, numOfBins + 1))
    plt.title("Exp " + str(i+1) + " histogram")
    plt.show()

##**Export to Excel**

In [ ]:
for i in range(X_test.shape[0]):
    np.savetxt("Exp_"+str(i+1)+"_histogram.csv", X_test[i], delimiter=',')
    np.savetxt("Exp_"+str(i+1)+"_localization.csv", clusterCoordinations[i], delimiter=',')

print("Data export completed successfully")

##**Visualize Localizations**
The next block will plot a max projection image of the last experiment with the localization marked on it.

In [ ]:
if(LoadData):
    debug_entire_exp(Max_Data_Set, coordinates, scale_size)

##**Data Simulation Configuration**
If you chose to simulate the training data, you would need to specify the following parameters:

**numOfClusters** - Specifies the number of simulated clusters in each simulation (relevant only if CreateSimulatedData is set to True).

**bleach_proba** - Specifies the bleaching probability [0, 1] of the used fluorophore.

**TrainSetSize** - Specifies the number of simulated experiments to be created.

In [ ]:
numOfClusters = 200 #@param {type:"number"}
bleach_proba = 0.41 #@param {type:"number"}
TrainSetSize = 10000 #@param {type:"number"}

##**Create Simulated Training Data**


In [ ]:
if(CreateSimulatedData):
    [X, y] = CreateSimulatedDataSet(TrainSetSize, numOfClusters, bleach_proba, numOfBins)
else:
    [X, y] = LoadSimulatedDataSet()

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.75)
[X_train, X_val, X_test] = Normalization(X_train, X_val, X_test)
[X_train, X_val, X_test] = BiasTrick(X_train, X_val, X_test)
y_train = torch.FloatTensor(y_train)
y_val = torch.FloatTensor(y_val)

print("-I- Simulated Data was created successfully")

##**Build Model**
In the next block we will build the neural network model.

**lr** - Specifies the training phase learning rate.

**betas** - Specifies the parameters for ADAM optimizer.

**batch_size** - Specifies the batch size of the training phase.

**epochs** - Specifies the maximal training epoch.

**early_stopping** - Specifies the tolerance of the neural network to lack of improvement in the validation loss. For example: early_stopping = 5, would stop the trainig phase if the validation loss did not improve for 5 epochs.

In [ ]:
lr = 1e-5 #@param {type:"number"}
betas = (0.99, 0.999) 
batch_size = 4 #@param {type:"number"}
epochs = 1000 #@param {type:"number"}
early_stopping = np.min((int(epochs/5), 15))

##**Training Phase**


In [ ]:
if(TrainNet):
    model = CustomNet(torch.numel(X_train[0]), [128, 128, 128, 128])
    
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=betas)

    dl_train = CreateDataLoader(X_train, y_train, batch_size=batch_size)
    dl_val = CreateDataLoader(X_val, y_val, batch_size=1)

    # ================= Train Net ================
    trainer = Trainer(model, criterion, optimizer)
    trainer.fit(dl_train, dl_val, num_epochs=epochs, early_stopping=early_stopping, print_every=1)
    torch.save(trainer.model.state_dict(), 'model_final_gauss')

##**Load Pre-trained Model**


In [ ]:
model = CustomNet(torch.numel(X_train[0]), [128, 128, 128, 128])
model.load_state_dict(torch.load(preTrainedModel))
print("Pretrained model loaded successfully")

##**Testing Phase**

In [ ]:
y_val_pred = model(X_val)
y_test_pred = model(X_test).squeeze()
y_test_pred = torch.max(y_test_pred, torch.zeros(y_test_pred.shape))

val_acc = torch.mean(torch.abs(y_val_pred.squeeze() - y_val))
print("Neural Network Validation MSE:", 100 * val_acc.item())

print("Printing dimer percentage per experiment:")
if(y_test_pred.shape == torch.Size([])):
    print("1: ", 100 * y_test_pred.item())
else:
    for i in range(y_test_pred.shape[0]):
        print(str(i+1)+": ", 100 * y_test_pred[i].item())

##**Detection Efficiency Correction**
Please specify the detection efficiency in your experiment. It should be a number in the range [0,1].

In [ ]:
detection_efficiency = 0.78 #@param {type:"number"}


###**Calculate Final Predictions**

In [ ]:
print("Printing corrected dimer percentage per experiment:")
if(y_test_pred.shape == torch.Size([])):
    print("1: ", 100 * find_actual_dimers_percentage(y_test_pred.item(), detection_efficiency))
else:
    for i in range(y_test_pred.shape[0]):
         print(str(i+1)+": ", 100 * find_actual_dimers_percentage(y_test_pred[i].item(), detection_efficiency))